In [ ]:
!git clone https://github.com/kakao/khaiii.git

!pip install cmake

!mkdir build

!cd build && cmake /content/khaiii

!cd /content/build/ && make all

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python


fatal: destination path 'khaiii' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
-- [hunter] Calculating Config-SHA1
-- [hunter] HUNTER_ROOT: /root/.hunter
-- [hunter] [ Hunter-ID: 70287b1 | Toolchain-ID: 02ccb06 | Config-ID: dffbc08 ]
-- [hunter] BOOST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.68.0-p1)
-- Boost version: 1.68.0
-- [hunter] CXXOPTS_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 2.1.1-pre)
-- [hunter] EIGEN_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.5)
-- [hunter] FMT_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 4.1.0)
-- [hunter] GTEST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.8.0-hunter-p11)
-- [hunter] NLOHMANN_JSON_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.0)
-- [hunter] SPDLOG_ROOT: /root/.hunte

In [ ]:
from khaiii import KhaiiiApi
api = KhaiiiApi()
for word in api.analyze("이거 되냐? 되ㅐ냐고"):
    for morph in word.morphs:
        print(morph.lex)


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
#주요 참고 PyTorch로 시작하는 딥 러닝 입문, 유원준
from torchtext import data  
import urllib.request
import pandas as pd
import random

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def isAll0(x):
    if type(x) == float:
        return x
    elif len(x) == x.count(' '):
        return ''
    else:
        return x


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")


train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')


train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df['document'] = train_df['document'].apply(isAll0)
train_df['document'].replace('', np.nan, inplace=True)


test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['document'] = test_df['document'].apply(isAll0)
test_df['document'].replace('', np.nan, inplace=True)


train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)

train_df = train_df.dropna(how = 'any')
test_df = test_df.dropna(how = 'any')

In [ ]:
print(len(train_df))
print(len(test_df))

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
train_x = []

In [ ]:
j = 0
for i, row in train_df.iterrows():
    val = row['document']
    j += 1
    if j % 10000 == 0:
        print(i,"/","143660")

    temp = ['CLS']
    for word in api.analyze(val):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                temp.append(morph.lex)
    temp.append('SEP')
    train_x.append(temp)


In [ ]:
test_x = []

In [ ]:
j = 0
for i, row in test_df.iterrows():
    val = row['document']
    j += 1
    if i % 10000 == 0:
        print(j,"/","48000")
    temp = ['CLS']
    
    for word in api.analyze(val):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                temp.append(morph.lex)
    temp.append('SEP')
    test_x.append(temp)


In [ ]:
vocab_size = 10000

tokenizer = Tokenizer(vocab_size,oov_token = 'OOV')
tokenizer.fit_on_texts(train_x)

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [ ]:
train_y = np.array(train_df['label'])
test_y = np.array(test_df['label'])

In [ ]:
drop_train = [index for index, sentence in enumerate(train_x) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(test_x) if len(sentence) < 1]

In [ ]:
train_x = np.delete(train_x, drop_train, axis=0)
train_y = np.delete(train_y, drop_train, axis=0)
print(len(train_x))
print(len(train_y))

In [ ]:
test_x = np.delete(test_x, drop_test, axis=0)
test_y = np.delete(test_y, drop_test, axis=0)
print(len(test_x))
print(len(test_y))

In [ ]:
print(train_x)

In [ ]:
pad_len = 30
train_x = pad_sequences(train_x, maxlen = pad_len,padding='post')
test_x = pad_sequences(test_x, maxlen = pad_len,padding='post')

In [ ]:
print(train_x)
print(train_y)

In [ ]:
class nlp_dataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x[idx])
        y = torch.FloatTensor(self.y[idx])
        return x, y

In [ ]:
class grubase(nn.Module):
    def __init__(self, embed_dim, vocab_size, hidden_dim, num_layers, batch_size, dropout):
        super(grubase, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.relu = nn.ReLU()
        self.batch_size = batch_size
        self.sigmoid = nn.Sigmoid()


        self.norm = nn.BatchNorm1d(self.batch_size)
        self.embed = nn.Embedding(self.vocab_size,self.embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.gru = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers,batch_first=True)

        self.mlp1 = nn.Linear(self.hidden_dim,1)

    def forward(self,x):
        x = self.embed(x)
        x = self.dropout(x)

        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        x = x[:,-1,:]
        x = self.norm(x)
        #x = torch.cat((x[:,0,:],x[:,-1,:]),dim=-1)

        x = self.dropout(x)
        x = self.mlp1(x)
        #x = self.sigmoid(x).squeeze()
        return x.squeeze()


In [ ]:
def train(model, optimizer, loss_function,train_loader,DEVICE):
    model.train()
    for batch in train_loader:
        x, y = batch
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        optimizer.zero_grad()

        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        loss.backward()
        optimizer.step()

In [ ]:
def getF1(y_pred,y,threshold=0.5):
    
    yp = [1 if x > threshold else 0 for x in y_pred]

    pp = 0
    pf = 0
    fp = 0
    ff = 0
    for i in range(len(y)):
        if y[i] > threshold:
            if yp[i] > threshold: pp += 1
            else: pf += 1
        else:
            if yp[i] < threshold: ff += 1
            else: fp += 1

    precision = pp / (pp + fp + 1e-5) 
    recall = pp / (pp + ff + 1e-5)
    F1 = 2 * precision * recall / (precision + recall + 1e-5)
    acc = (pp + ff) / (len(y) + 1e-5)
    return F1, acc

In [ ]:
def evaluate(model, val_loader, loss_function, DEVICE, batch_size, threshold):
    """evaluate model"""
    model.eval()
    total_loss = 0
    total_f1 = 0
    total_acc = 0
    for batch in val_loader:
        x, y = batch
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        f1, acc = getF1(y_pred,y,threshold)
        total_f1 += f1
        total_acc += acc
        total_loss += loss.item()

    size = len(val_loader) / batch_size
    avg_loss = total_loss / size
    avg_f1 = total_f1 / size
    avg_acc = total_acc / size
    return avg_loss, avg_f1, avg_acc

In [ ]:
device = torch.device("cuda")

batch_size = 256
embed_dim = 128
hidden_dim = 256
dropout = 0.3
layers = 1

model = grubase(embed_dim,vocab_size+2,hidden_dim,layers,batch_size,dropout)
model.to(device)
loss = nn.BCEWithLogitsLoss(pos_weight = 1.1 * torch.ones([1])).to(device)
lr = 0.003
threshold = 0.5

EPOCHS = 20
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
train_dataset = nlp_dataset(train_x,train_y)
test_dataset = nlp_dataset(test_x,test_y)

train_loader = DataLoader(train_dataset,batch_size,True)
val_loader = DataLoader(test_dataset,batch_size,True)

In [ ]:
for i in train_loader:
    x, y = i
    print(x,y)
    print(len(x))
    print(len(y))
    break

In [ ]:
best_val_f1 = 0
for e in range(1, EPOCHS+1):
    train(model, optimizer, loss,train_loader,device)
    val_loss,val_f1,val_acc = evaluate(model, val_loader, loss, device,batch_size,threshold)

    print("[Epoch: %d] val loss : %1.5f    val acc :%4.3f    F1 :%4.3f" % (e, val_loss, val_acc,val_f1))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_f1 or val_f1 > best_val_f1:
        print("Best saved")
        torch.save(model.state_dict(), '/content/gdrive/My Drive/GRUmodel/NLP_esemble_model.pt')
        best_val_f1 = val_f1